In [1]:
from nltk.probability import FreqDist
import json
import numpy as np

### Loading data from unzipped file

In [2]:
# Attention: You need to unzip "reviews_Home_and_Kitchen_5.json" to get "Home_and_Kitchen_5.json"
# set USE SMALL SUBSET if using sample data
USE_SMALL_SUBSET = False
path = "data.json" if USE_SMALL_SUBSET else "Home_and_Kitchen_5.json"

In [3]:
with open(path, encoding='utf-8') as f:
    data = [json.loads(line) for line in f.readlines()]

In [4]:
len(data)

551682

In [5]:
""" IGNORE WHEN running from data json
with open('data.json', 'w') as outfile:
        for i in data[:100_000]:
            json.dump(i, outfile)
            outfile.write("\n")
"""

' IGNORE WHEN running from data json\nwith open(\'data.json\', \'w\') as outfile:\n        for i in data[:100_000]:\n            json.dump(i, outfile)\n            outfile.write("\n")\n'

In [6]:
# proccessed_data = []
proccessed_data = [ [dics["reviewText"], dics["helpful"], dics['overall'] ] for dics in data]
# for dics in data:
#     line = []
#     line.append(dics["reviewText"])
#     line.append(dics["helpful"])
#     line.append(dics['overall'])
#     proccessed_data.append(line)

Checking number of unique words in the data set before preproccessing:

##  Filtering Punctuation

In [7]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
punct_set = set(punctuation)

In [9]:
def filter_punctuation(text):
    no_punct=[words for words in text if words not in punct_set]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

In [10]:
for data in proccessed_data:
    data[0] = filter_punctuation(data[0])

In [11]:
fdist_wo_punct = FreqDist()
for line in proccessed_data:
    for word in line[0].split():
        fdist_wo_punct[word.lower()] += 1
print(fdist_wo_punct)

<FreqDist with 481174 samples and 53739451 outcomes>


number of unique words went down to 481174 from 838417 after removing the puncuation :)

## Tokenizing

In [12]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gaston\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
for data in proccessed_data:
    data[0] = word_tokenize(data[0])

In [14]:
fdist_wo_punct.most_common(50)

[('the', 2780773),
 ('i', 1606253),
 ('and', 1557137),
 ('a', 1438350),
 ('to', 1425398),
 ('it', 1387669),
 ('is', 876047),
 ('of', 835434),
 ('this', 759706),
 ('for', 728397),
 ('in', 612086),
 ('that', 551863),
 ('my', 467378),
 ('with', 446288),
 ('you', 432856),
 ('have', 428896),
 ('on', 425902),
 ('but', 399901),
 ('not', 355774),
 ('so', 322844),
 ('as', 308031),
 ('was', 305702),
 ('are', 304115),
 ('one', 270744),
 ('use', 243819),
 ('they', 241978),
 ('be', 239917),
 ('its', 239749),
 ('very', 232904),
 ('or', 218627),
 ('like', 202988),
 ('just', 200785),
 ('if', 194194),
 ('up', 193151),
 ('out', 187099),
 ('can', 183018),
 ('great', 179985),
 ('at', 177328),
 ('these', 176098),
 ('when', 175176),
 ('all', 171446),
 ('had', 166292),
 ('them', 164924),
 ('well', 161751),
 ('would', 161676),
 ('will', 158631),
 ('more', 147408),
 ('good', 145571),
 ('from', 144893),
 ('than', 139689)]

## Filtering Stop Words: 

In [15]:
from nltk.corpus import stopwords

In [16]:
stop_words = set(stopwords.words('english'))

In [17]:
def remove_stopwords(text):
    text=[word for word in text if word.lower() not in stop_words]
    return text

In [18]:
for data in proccessed_data:
    data[0] = remove_stopwords(data[0])

### Creating scaling values for helpfulness:

In [19]:
def create_sntmtl_label(help_list):
    if help_list[1] == 0:
        return 1.0
#         return "neutral"
    return 1+(1.1*(help_list[0] / help_list[1]))

In [20]:
for data in proccessed_data:
    data[1] = create_sntmtl_label(data[1])

## creating sentimental labels for overall ratings:

In [21]:
def create_rating_sntmtl(rating: float):
    if rating <=3:
        return 0
    else:
        return 1

In [22]:
for line in proccessed_data:
    line[2] = create_rating_sntmtl(line[2])

## Creating a Panda dataframe:

In [23]:
import pandas as pd
from pandas import DataFrame

In [24]:
df = DataFrame(proccessed_data, columns=['Text','Helpfulness','Rating'])

In [25]:
df.head()

,Text,Helpfulness,Rating
0,"[daughter, wanted, book, price, Amazon, best, ...",1.000000,1
1,"[bought, zoku, quick, pop, daughterr, zoku, qu...",1.000000,1
2,"[shortage, pop, recipes, available, free, web,...",2.059259,1
3,"[book, must, get, Zoku, also, highly, recommen...",1.855556,1
4,"[cookbook, great, really, enjoyed, reviewing, ...",1.000000,1


In [26]:
df.to_csv("Tokenized.csv", index=False)

In [27]:
SEED = 1234

In [28]:
import pandas as pd
import ast

In [29]:
df= pd.read_csv("Tokenized.csv")
df.head()

,Text,Helpfulness,Rating
0,"['daughter', 'wanted', 'book', 'price', 'Amazo...",1.000000,1
1,"['bought', 'zoku', 'quick', 'pop', 'daughterr'...",1.000000,1
2,"['shortage', 'pop', 'recipes', 'available', 'f...",2.059259,1
3,"['book', 'must', 'get', 'Zoku', 'also', 'highl...",1.855556,1
4,"['cookbook', 'great', 'really', 'enjoyed', 're...",1.000000,1


In [30]:
df.shape

(551682, 3)

In [31]:
df["Text"]= df["Text"].map(lambda x: ast.literal_eval(x))

In [32]:
from sklearn.model_selection import train_test_split
#split is 60 20 20
X_train, X_test, __, _ = train_test_split(df[['Text',"Helpfulness","Rating"]], df[['Rating']], test_size=0.20, random_state=SEED)

In [33]:
X_train, X_val, __, _ = train_test_split(X_train, __, test_size=0.25, random_state=SEED)

In [34]:
X_train.to_csv("./data/Train.csv",index=False)
X_val.to_csv("./data/Val.csv",index=False)
X_test.to_csv("./data/Test.csv",index=False)